# Optimizing Dataframes and Processing in Chunks

## Introduction

In this project, we will be working with a dataset of Lending Club loans that were approved from 2007-2011. Lending Club is a marketplace for personal loans that matches borrowers with investors.

The goal of this project is to practice working with chunked dataframes and optimizing a dataframe's memory usage. The provided dataset consumes about 67 MB of memory. We will assume that we only have 10 MB of memory available in this project.

In [1]:
import pandas as pd
pd.options.display.max_columns = 99

Let's start off by reading the first five lines from loans_2007.csv to look for any data quality issues.

In [2]:
first_five_lines = pd.read_csv('loans_2007.csv', nrows=5)
first_five_lines.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


The data in the first five rows looks fine. Let's now read in the first 1000 rows from the data set and calculate the total memory usage for these rows. We will adjust the number of rows that we read in until we get the memory usage right under 5 MB (50% of our available memory).

In [3]:
first_1000_rows = pd.read_csv('loans_2007.csv', nrows=1000)
first_1000_rows.memory_usage(deep=True).sum() / (2 ** 20)

1.5502548217773438

Calling DataFrame.memory_usage(deep=True).sum() gives us the memory footprint of the dataframe in bytes. We are looking to keep the footprint under 5 MB, so we divided by 2 ** 20 to convert from bytes to MB. We can see that 1000 rows takes up 1.55 MB, so we can increase by a factor of 3 and stay under the 5 MB limit.

In [4]:
first_3000_rows = pd.read_csv('loans_2007.csv', nrows=3000)
first_3000_rows.memory_usage(deep=True).sum() / (2 ** 20)

4.649059295654297

We can see that 3000 rows takes up about 4.65 MB. As a result, we will use a chunksive of 3000 rows.

Let's write a function that optimizes this process: Determine the optimal chunk size based on the memory constraints you provide.

In [33]:
def find_chunk_size(file, initial_num_rows, mem_limit):
    rows = pd.read_csv(file, nrows=initial_num_rows)
    rows_mem_footprint = rows.memory_usage(deep=True).sum()
    avg_row_size = rows_mem_footprint / initial_num_rows
    return int(mem_limit / avg_row_size)

print(find_chunk_size('loans_2007.csv', 1000, 5000000))

3075


## Exploring the Data in Chunks

For each chunk, we will look at the following questions:
* How many columns have a numeric type? How many have a string type? 
* How many unique values are there in each string column? How many of the string columns have less than 50 unique values?
* Which float columns have no missing values and could be candidates for conversion to the integer type?

In [5]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000
)

for chunk in chunk_iter:
    numeric_columns = chunk.select_dtypes(include=['int', 'float']).columns
    string_columns = chunk.select_dtypes(include=['object']).columns
    print("Number of numeric columns:", len(numeric_columns))
    print("Numeric columns:", numeric_columns)
    print("Number of string columns:", len(string_columns))
    print("String columns:", string_columns)

Number of numeric columns: 31
Numeric columns: Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_liens'],
      dtype='object')
Number of string columns: 21
String columns: Index(['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d',
       'last_cre

Number of numeric columns: 31
Numeric columns: Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_liens'],
      dtype='object')
Number of string columns: 21
String columns: Index(['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d',
       'last_cre

We see something strange above, where for most of the chunks, there are 31 numeric columns and 21 string columns, but for the last 2 chunks, there are 30 numeric columns and 22 string columns. We see that it is because the 'id' column is viewed as an object column for some reason. We will ignore the 'id' column because it does not provide any useful information from an analysis standpoint.

Now we will find how many unique values there are in each string column, and see if any have less than 50 unique values.

In [6]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000
)

string_col_unique_vals = {}

for chunk in chunk_iter:
    string_columns = chunk.select_dtypes(include=['object']).columns
    
    for string_col_name in string_columns:
        unique_vals = chunk[string_col_name].value_counts()
        if string_col_name not in string_col_unique_vals:
            string_col_unique_vals[string_col_name] = unique_vals
        else:
            string_col_unique_vals[string_col_name] = pd.concat([string_col_unique_vals[string_col_name], unique_vals])

for val in string_col_unique_vals:
    vc = string_col_unique_vals[val]
    vc_grouped = vc.groupby(vc.index).sum()
    string_col_unique_vals[val] = len(vc_grouped)
    
print(string_col_unique_vals)

{'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_title': 30658, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'title': 21264, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1, 'id': 3538}


In [7]:
num_str_col_lt_50_unique = 0
cols_to_convert_to_cat = []

for key, val in string_col_unique_vals.items():
    if val < 50:
        num_str_col_lt_50_unique += 1
        cols_to_convert_to_cat.append(key)
        
print("Number of string columns with less than 50 unique values:", num_str_col_lt_50_unique)

Number of string columns with less than 50 unique values: 11


In [8]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000
)

missing = []

for chunk in chunk_iter:
    numeric_columns = chunk.select_dtypes(include=['float'])
    missing.append(numeric_columns.apply(pd.isnull).sum())

total_missing_values = pd.concat(missing)
total_missing_values = total_missing_values.groupby(total_missing_values.index).sum().sort_values()
print("Missing values for each float column:", total_missing_values)

Missing values for each float column: member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mt

None of the float columns have no missing values.

Let's calculate the total memory usage across all of the chunks.

In [9]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000
)

memory_footprint = 0

for chunk in chunk_iter:
    memory_footprint += chunk.memory_usage(deep=True).sum()

print("Total memory footprint (bytes):", memory_footprint)

Total memory footprint (bytes): 69432571


## Optimizing String Columns

We can save a lot of memory by converting string columns to numeric category types. We will first check if any of our string columns can easily be converted to numerical columns. Next, we will do this for all the columns in our cols_to_convert_to_cat list that we calculated earlier, which contains all the columns with less than 50 unique values.

In [10]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000
)

for chunk in chunk_iter:
    string_columns = chunk.select_dtypes(include=['object'])
    print(string_columns)

            term int_rate grade sub_grade                 emp_title  \
0      36 months   10.65%     B        B2                       NaN   
1      60 months   15.27%     C        C4                     Ryder   
2      36 months   15.96%     C        C5                       NaN   
3      36 months   13.49%     C        C1       AIR RESOURCES BOARD   
4      60 months   12.69%     B        B5  University Medical Group   
...          ...      ...   ...       ...                       ...   
2995   36 months    6.62%     A        A2   US Department of Energy   
2996   36 months   12.42%     B        B4                  Best Buy   
2997   36 months   16.77%     D        D2          Northrop Grumman   
2998   60 months   12.42%     B        B4  Commonwealth of Virginia   
2999   36 months    6.03%     A        A1                BNY Mellon   

     emp_length home_ownership verification_status   issue_d  loan_status  \
0     10+ years           RENT            Verified  Dec-2011   Fully P

             term int_rate grade sub_grade                       emp_title  \
9000    60 months   18.39%     E        E2  Effingham County Middle School   
9001    36 months   10.99%     B        B3       Catalyst Health Solutions   
9002    60 months   13.49%     C        C2     Centers For Disease Control   
9003    36 months    5.42%     A        A1                Valencia College   
9004    36 months   11.49%     B        B4                 Bang Salon Inc.   
...           ...      ...   ...       ...                             ...   
11995   36 months   12.99%     C        C1                            USMC   
11996   36 months   11.99%     B        B5                      IAMAW DL65   
11997   36 months   11.99%     B        B5                General Electric   
11998   36 months   17.49%     D        D5           LitePoint Corporation   
11999   36 months    6.99%     A        A3       Bureau of Land Management   

      emp_length home_ownership verification_status   issue_d l

             term int_rate grade sub_grade                    emp_title  \
21000   36 months    7.29%     A        A4                     Wachovia   
21001   36 months    5.79%     A        A2        Services at Brookside   
21002   36 months   10.74%     B        B4  Pacific Marine Credit Union   
21003   36 months   16.40%     E        E1        Spartan Solutions Inc   
21004   36 months   14.54%     D        D1                     Bernards   
...           ...      ...   ...       ...                          ...   
23995   36 months   12.61%     C        C2     Sunnrise Asiisted Living   
23996   60 months   12.61%     C        C2         Honeywell Intl. Inc.   
23997   60 months    6.91%     A        A5     Dubois-Cooper Associates   
23998   36 months   14.46%     D        D2                          NaN   
23999   60 months    6.91%     A        A5        Protech Solutions Inc   

      emp_length home_ownership verification_status   issue_d  loan_status  \
21000   < 1 year     

             term int_rate grade sub_grade  \
33000   36 months   12.18%     B        B4   
33001   36 months   16.00%     D        D5   
33002   36 months   11.83%     B        B3   
33003   36 months   12.18%     B        B4   
33004   36 months   11.83%     B        B3   
...           ...      ...   ...       ...   
35995   36 months   13.16%     C        C3   
35996   36 months    8.00%     A        A3   
35997   36 months   12.21%     B        B5   
35998   36 months   12.84%     C        C2   
35999   36 months    7.68%     A        A2   

                                   emp_title emp_length home_ownership  \
33000             General Casualty Insurance    4 years           RENT   
33001  Governor's Office of Customer Service    2 years           RENT   
33002                        EMC Corporation    3 years           RENT   
33003                               Wal-Mart    6 years           RENT   
33004                     Marshalls Cleaners    8 years       MORTGAGE   
...

We can see above that there are 2 string columns with percentage values that are only string data type because of the % character at the end: int_rate, revol_util

We also see that the term column is just a number appended with "months".

For these 3 columns, we will just remove the non-numerical characters and  rename the columns to:
int_rate (%), revol_util (%), and term (months).

We also have the following columns that we can convert to category columns:

In [11]:
print(cols_to_convert_to_cat)

['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 'initial_list_status', 'application_type']


We are already converting term to a numerical column, so let's remove it from this list.

In [12]:
cols_to_convert_to_cat = cols_to_convert_to_cat[1:]

Lastly, a few of the columns are dates, so we can parse them as dates instead of strings: issue_d, earliest_cr_line, last_pymnt_d, and last_credit_pull_d.

Let's now perform these optimizations on the chunks and calculate the new memory footprint.

In [27]:
# Helper Function for optimizing String columns
def convert_string_col_to_numerical(chunk):
    # Stripping the non-numerical characters
    chunk['int_rate'] = chunk['int_rate'].str.lstrip(" ").str.rstrip('%')
    chunk['revol_util'] = chunk['revol_util'].str.lstrip(" ").str.rstrip('%')
    chunk['term'] = chunk['term'].str.lstrip(" ").str.rstrip(' months')
    
    # Renaming the columns
    chunk.rename(columns={
        "int_rate": "int_rate (%)",
        "revol_util": "revol_util (%)",
        "term": "term (months)"
    })
    
def convert_string_col_to_categorical(cols):
    # Converting other columns to category-type columns
    for col in cols:
        chunk[col] = chunk[col].astype("category")
        
def optimize_string_cols(chunk, category_cols):
    convert_string_col_to_numerical(chunk)
    convert_string_col_to_categorical(category_cols)

In [13]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000,
    parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
)

memory_footprint = 0

for chunk in chunk_iter:
    optimize_string_cols(chunk, cols_to_convert_to_cat)
    memory_footprint += chunk.memory_usage(deep=True).sum()
    
print("Memory footprint (bytes) after optimizing string columns:", memory_footprint)

Memory footprint (bytes) after optimizing string columns: 31777537


We can see that with the above optimizations to string columns, we were able to cut down the memory footprint of the dataset by over half.

## Optimizing Numeric Columns

We will continue to optimize the memory footprint of our dataframe by working with the float columns. We will:

* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.

Based on our findings above, we will perform the necessary type changes across all chunks, then calculate the total memory footprint.

We saw earlier that none of our float columns had no missing values, so we can skip the second bullet point (see below)

In [14]:
print(total_missing_values)

member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

For each of these float columns, we will find the minimum and maximum, and based on our findings, will determine if we can downcast to a float type with a smaller memory footprint.

In [17]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000,
    parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
)

max_val_dict = {}
min_val_dict = {}

for chunk in chunk_iter:
    for col in chunk.select_dtypes(include=['float']).columns:
        max_val = chunk[col].max()
        min_val = chunk[col].min()
        if col in max_val_dict:
            if max_val > max_val_dict[col]:
                max_val_dict[col] = max_val
        else:
            max_val_dict[col] = max_val
        if col in min_val_dict:
            if min_val < min_val_dict[col]:
                min_val_dict[col] = min_val
        else:
            min_val_dict[col] = min_val

{'member_id': 1314167.0, 'loan_amnt': 35000.0, 'funded_amnt': 35000.0, 'funded_amnt_inv': 35000.0, 'installment': 1305.19, 'annual_inc': 6000000.0, 'dti': 29.99, 'delinq_2yrs': 13.0, 'inq_last_6mths': 33.0, 'open_acc': 47.0, 'pub_rec': 5.0, 'revol_bal': 1207359.0, 'total_acc': 90.0, 'out_prncp': 5794.29, 'out_prncp_inv': 5794.29, 'total_pymnt': 58563.6799293133, 'total_pymnt_inv': 58563.68, 'total_rec_prncp': 35000.02, 'total_rec_int': 23611.1, 'total_rec_late_fee': 209.000000006491, 'recoveries': 29623.35, 'collection_recovery_fee': 7002.19, 'last_pymnt_amnt': 36115.2, 'collections_12_mths_ex_med': 0.0, 'policy_code': 1.0, 'acc_now_delinq': 1.0, 'chargeoff_within_12_mths': 0.0, 'delinq_amnt': 6053.0, 'pub_rec_bankruptcies': 2.0, 'tax_liens': 1.0}
{'member_id': 70473.0, 'loan_amnt': 500.0, 'funded_amnt': 500.0, 'funded_amnt_inv': 0.0, 'installment': 15.67, 'annual_inc': 1896.0, 'dti': 0.0, 'delinq_2yrs': 0.0, 'inq_last_6mths': 0.0, 'open_acc': 1.0, 'pub_rec': 0.0, 'revol_bal': 0.0, 'to

We know that in pandas, we have float16, float32, and float64. Here are the ranges of these datatypes:

In [25]:
import numpy as np
fi16_min = np.finfo(np.float16).min
fi16_max = np.finfo(np.float16).max
fi32_min = np.finfo(np.float32).min
fi32_max = np.finfo(np.float32).max
fi64_min = np.finfo(np.float64).min
fi64_max = np.finfo(np.float64).max

print("Minimum of float16:", fi16_min)
print("Maximum of float16:", fi16_max)
print("Minimum of float32:", fi32_min)
print("Maximum of float32:", fi32_max)
print("Minimum of float64:", fi64_min)
print("Maximum of float64:", fi64_max)

Minimum of float16: -65500.0
Maximum of float16: 65500.0
Minimum of float32: -3.4028235e+38
Maximum of float32: 3.4028235e+38
Minimum of float64: -1.7976931348623157e+308
Maximum of float64: 1.7976931348623157e+308


In [28]:
# Helper Function for downcasting float columns
def downcast_float_cols(chunk):
    for col in chunk.select_dtypes(include=['float']).columns:
        max_val = max_val_dict[col]
        min_val = min_val_dict[col]
        if max_val <= fi16_max and min_val >= fi16_min:
            chunk[col] = chunk[col].astype('float16')
        elif max_val <= fi32_max and min_val >= fi32_min:
            chunk[col] = chunk[col].astype('float32')
        else:
            chunk[col] = chunk[col].astype('float64')

In [31]:
chunk_iter = pd.read_csv(
    'loans_2007.csv', 
    chunksize=3000,
    parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
)

memory_footprint = 0

for chunk in chunk_iter:
    optimize_string_cols(chunk, cols_to_convert_to_cat)
    downcast_float_cols(chunk)
    memory_footprint += chunk.memory_usage(deep=True).sum()
    
print("Memory footprint (bytes) after optimizing string and numeric columns:", memory_footprint)

Memory footprint (bytes) after optimizing string and numeric columns: 24375925


After optimizing the string and numeric columns, we managed to take a dataset with a memory footprint of 69432571 bytes (66.21 MB) and trimmed it to a third of the original size: 24375925 bytes (23.2467 MB).